# Processing cellpy batch - cycles

### `{{cookiecutter.project_name}}::{{cookiecutter.session_id}}`

**Experimental-id:** `{{cookiecutter.notebook_name}}`  
**Short-name:** `{{cookiecutter.session_id}}`  
**Project:** `{{cookiecutter.project_name}}`  
**By:** `{{cookiecutter.author_name}}`   
**Date:** `{{cookiecutter.date}}`  

## Imports and setup

In [ ]:
import os
import pathlib

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import holoviews as hv
import hvplot.pandas
from holoviews import opts

import cellpy
from cellpy import prms
from cellpy import prmreader
from cellpy.utils import batch, helpers, plotutils

In [ ]:
%matplotlib inline
hv.extension('bokeh')
pd.set_option('display.max_columns', 70)

In [ ]:
cellpy_batch_file = "cellpy_batch_" + "{{cookiecutter.session_id}}" + ".json"

In [ ]:
b = batch.init(default_log_level="INFO", db_reader=None)
b.experiment.journal.from_file(cellpy_batch_file)
b.link()
cell_labels = b.experiment.cell_names
print(" cell labels: ")
cell_labels

## Plot all cycles

In [ ]:
voltage_curves = dict()
for label in b.experiment.cell_names:
    d = b.experiment.data[label]
    curves = d.get_cap(label_cycle_number=True, interpolated=True, number_of_points=100)
    if not curves.empty:
        curve = hv.Curve(curves, kdims=["capacity", "cycle"], vdims="voltage").groupby("cycle").overlay()#.opts(show_legend=False)
        voltage_curves[label] = curve
    else:
        print(f"[{label} empty]")
NdLayout_voltage_curves = hv.NdLayout(voltage_curves, kdims='label').cols(3)

In [ ]:
palette = 'Spectral'
NdLayout_voltage_curves.opts(
    hv.opts.Curve(color=hv.Palette(palette), tools=['hover']),
    hv.opts.NdOverlay(shared_axes=False),
    hv.opts.NdLayout(),
)

## Links

### Notebooks
- notes and information [link](00_{{cookiecutter.notebook_name}}_notes.ipynb)
- processing raw data [link](01_{{cookiecutter.notebook_name}}_loader.ipynb)
- life [link](02_{{cookiecutter.notebook_name}}_life.ipynb)
- cycles [link](03_{{cookiecutter.notebook_name}}_cycles.ipynb)
- ica [link](04_{{cookiecutter.notebook_name}}_ica.ipynb)
- plots [link](05_{{cookiecutter.notebook_name}}_plots.ipynb)